# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time


from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data = pd.read_csv("../Resources/weather_data.csv")
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Geraldton,2,AU,1616494901,64,-28.7667,114.6000,71.60,21.85
1,Waipawa,100,NZ,1616494902,88,-41.4122,175.5153,55.40,1.14
2,Ushuaia,90,AR,1616494808,87,-54.8000,-68.3000,44.60,5.50
3,Hithadhoo,10,MV,1616494903,68,-0.6000,73.0833,83.77,7.49
4,Belmonte,20,BR,1616494904,94,-15.8631,-38.8828,73.40,4.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [6]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
27,Lady Frere,0,ZA,1616494923,22,-31.7031,27.2329,79.12,7.76
70,Atar,0,MR,1616494957,13,20.5169,-13.0499,71.60,9.22
83,Conceição do Araguaia,0,BR,1616494967,94,-8.2578,-49.2647,73.40,1.23
157,Arraial do Cabo,0,BR,1616495025,88,-22.9661,-42.0278,77.00,8.05
196,Mauá,0,BR,1616495059,78,-23.6678,-46.4614,73.00,1.14
209,Acapulco de Juárez,0,MX,1616494967,64,16.8634,-99.8901,72.23,4.74
232,Astorga,0,BR,1616495091,88,-23.2325,-51.6656,73.40,5.53
259,Qarqīn,0,AF,1616495112,30,37.4185,66.0436,73.44,4.27
383,Vila Velha,0,BR,1616494928,94,-20.3297,-40.2925,75.99,2.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,Lady Frere,ZA,-31.7031,27.2329,
70,Atar,MR,20.5169,-13.0499,
83,Conceição do Araguaia,BR,-8.2578,-49.2647,
157,Arraial do Cabo,BR,-22.9661,-42.0278,
196,Mauá,BR,-23.6678,-46.4614,
209,Acapulco de Juárez,MX,16.8634,-99.8901,
232,Astorga,BR,-23.2325,-51.6656,
259,Qarqīn,AF,37.4185,66.0436,
383,Vila Velha,BR,-20.3297,-40.2925,


In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [11]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

print("-------End of Search-------")

Retrieving Results for Index 27: Lady Frere.
Missing field/result... skipping.
------------
Retrieving Results for Index 70: Atar.
Missing field/result... skipping.
------------
Retrieving Results for Index 83: Conceição do Araguaia.
Missing field/result... skipping.
------------
Retrieving Results for Index 157: Arraial do Cabo.
Missing field/result... skipping.
------------
Retrieving Results for Index 196: Mauá.
Missing field/result... skipping.
------------
Retrieving Results for Index 209: Acapulco de Juárez.
Missing field/result... skipping.
------------
Retrieving Results for Index 232: Astorga.
Missing field/result... skipping.
------------
Retrieving Results for Index 259: Qarqīn.
Missing field/result... skipping.
------------
Retrieving Results for Index 383: Vila Velha.
Missing field/result... skipping.
------------
-------End of Search-------


In [12]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,Lady Frere,ZA,-31.7031,27.2329,
70,Atar,MR,20.5169,-13.0499,
83,Conceição do Araguaia,BR,-8.2578,-49.2647,
157,Arraial do Cabo,BR,-22.9661,-42.0278,
196,Mauá,BR,-23.6678,-46.4614,
209,Acapulco de Juárez,MX,16.8634,-99.8901,
232,Astorga,BR,-23.2325,-51.6656,
259,Qarqīn,AF,37.4185,66.0436,
383,Vila Velha,BR,-20.3297,-40.2925,


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))